In [ ]:
"""
Methodology:
loop through timestamps of skyprobe measurements, for each one find a timestamp in the telescope data within +/- 1 
minute as well as an ASIVA image with a timestamp within +/- 1 minute. 
output a csv file with 5 columns as follows:
1 - skyprobe timestamp
2 - Skyprobe attenuation
3 - telescope Az
4 - Telescope Alt
5 - ASIVA Image Filename
"""

In [ ]:
"""File Parsing:
Skyprobe Data:
Column 1: Time in UNIX EPOCH
Column 4: Zeropoint (Attenuation)

Telescope Data:
Column 1: Time in UNIX Epoch
Column 2: Alt
Column 3: Az

ASIVA Images:
naming example: asiva202301251900.png
naming format: asivaYYYYMMDDHHMM
HH in military time
"""

In [45]:
import pandas as pd


DF_skyprobe = pd.read_csv("skyprobe_data.csv", low_memory=False)
DF_skyprobe.columns = ['Time', 'skip1', 'skip2', 'attenuation', 'skip3', 'skip4']
DF_skyprobe = DF_skyprobe.drop(columns = ['skip1', 'skip2', 'skip3', 'skip4'])
DF_telescope = pd.read_csv("tcs_data.csv", low_memory=False)
DF_telescope.columns = ['Time', 'Alt', 'Az', 'skip']
DF_telescope = DF_telescope.drop(columns = ['skip'])
DF = pd.merge_asof(DF_skyprobe, DF_telescope, on = "Time", direction="nearest", tolerance=60)
DF = DF.dropna()
print(DF.head)




<bound method NDFrame.head of                Time  attenuation      Alt        Az
0        1274851510        0.107  89.8103  279.5074
1        1274851809        0.155  67.7496   80.3481
2        1274851870        0.144  67.7496   80.3481
3        1274851929        0.559  67.7496   80.3481
4        1274851989        0.121  67.7496   80.3481
...             ...          ...      ...       ...
2125491  1694792290       -0.108  64.9583   48.1660
2125492  1694792350        0.151  64.9213   48.1046
2125493  1694792407       -0.183  64.8382   48.1806
2125494  1694792470        0.004  65.0426   47.9788
2125495  1694793600        0.004  64.9767   48.1581

[1647649 rows x 4 columns]>


In [46]:
#we now have a combined dataset of skyprobe data with corresponding Alt and Az values.
#Now we need to find ASIVA images with close timestamps and add the filenames to the dataframe
import glob

folder = "../images"
filenames = []
timestamps = []
for file in glob.glob(folder + '/*.png'):
    filedate = file.split('/')[2].split('asiva')[1].split('.')[-2]
    timestamp = datetime.timestamp(datetime.strptime(filedate, '%Y%m%d%H%M'))
    filenames.append(file)
    timestamps.append(int(timestamp))
    #convert filedate/time to UNIX. Add 10 hours for converting HST to UTC
DF_ASIVA = pd.DataFrame(timestamps)
DF_ASIVA.columns = ["Time"]
DF_ASIVA["filenames"] = filenames
DF_ASIVA = DF_ASIVA.sort_values(by="Time")
print(DF_ASIVA.head)


<bound method NDFrame.head of             Time                        filenames
1409  1674698400  ../images/asiva202301251900.png
1442  1674698460  ../images/asiva202301251901.png
1251  1674698520  ../images/asiva202301251902.png
1208  1674698580  ../images/asiva202301251903.png
868   1674698640  ../images/asiva202301251904.png
...          ...                              ...
247   1687949700  ../images/asiva202306280355.png
75    1687949760  ../images/asiva202306280356.png
107   1687949820  ../images/asiva202306280357.png
2742  1687949880  ../images/asiva202306280358.png
2701  1687949940  ../images/asiva202306280359.png

[2999 rows x 2 columns]>


In [47]:
#perform another merge_asof to add asiva filenames to the main dataframe
DF = pd.merge_asof(DF, DF_ASIVA, on = "Time", direction="nearest", tolerance=60)
DF = DF.dropna()
print(DF.head)

<bound method NDFrame.head of                Time  attenuation      Alt        Az  \
1580413  1674709930        2.621  66.4844  323.4849   
1594348  1681794911        0.667  87.9959  268.2851   
1594349  1681795030        0.379  89.1985  237.2593   
1594350  1681795090        0.342  89.1985  255.5949   
1594351  1681795150        0.387  89.1985  260.8347   
...             ...          ...      ...       ...   
1616200  1687949712        0.049  70.0681  324.3404   
1616201  1687949772        0.033  70.0681  324.3404   
1616202  1687949890        0.045  70.0681  323.3369   
1616203  1687949952        0.042  70.0681  323.3369   
1616204  1687950000        0.042  70.0681  323.3369   

                               filenames  
1580413  ../images/asiva202301252212.png  
1594348  ../images/asiva202304172215.png  
1594349  ../images/asiva202304172217.png  
1594350  ../images/asiva202304172218.png  
1594351  ../images/asiva202304172219.png  
...                                  ...  
1616200 

In [ ]:
"""
Now need to find the location within each ASIVA image and save a 12x12 kernel surrounding that location
along with a corresponding attenuation value
"""